<a href="https://colab.research.google.com/github/m3y54m/ut-nndl-course/blob/master/miniproj1_4_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D 

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

print(tf.VERSION)
print(tf.keras.__version__)

1.12.0
2.1.6-tf


In [2]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)

x_train shape: (50000, 32, 32, 3)


In [3]:
trainset_size = x_train.shape[0]
testset_size = x_test.shape[0]
batch_size = 100
steps_per_epoch = np.ceil(trainset_size / batch_size).astype('int')
num_classes = 10
epochs = 100
# input shape: (32, 32, 3)
input_shape = x_train.shape[1:]

data_augmentation = True
drop_out = False

print(trainset_size, 'train samples')
print(testset_size, 'test samples')

50000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary (one-hot) class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [5]:
tf.keras.backend.clear_session()

model = Sequential()

model.add(Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

if drop_out:
  print('Using dropout.')
  model.add(Dropout(0.25))
else:
  print('Not using dropout.')
  
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

if drop_out:
  model.add(Dropout(0.25))
  
model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

if drop_out:
  model.add(Dropout(0.25))
  
model.add(Flatten())
model.add(Dense(256, activation='relu'))

if drop_out:
  model.add(Dropout(0.25))
  
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Not using dropout.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0 

In [0]:
# Let's train the model using AdamOptimizer
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# Normalize input numbers
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [8]:
if not data_augmentation:
  print('Not using data augmentation.')
  results = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    results = model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        steps_per_epoch=steps_per_epoch,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/100
500/500 [==============================] - 38s 77ms/step - loss: 1.5889 - acc: 0.4208 - val_loss: 1.3888 - val_acc: 0.4900
Epoch 2/100
500/500 [==============================] - 37s 73ms/step - loss: 1.2125 - acc: 0.5701 - val_loss: 1.0539 - val_acc: 0.6277
Epoch 3/100
500/500 [==============================] - 37s 73ms/step - loss: 1.0546 - acc: 0.6253 - val_loss: 0.9909 - val_acc: 0.6545
Epoch 4/100
439/500 [=========================>....] - ETA: 4s - loss: 0.9609 - acc: 0.6617

KeyboardInterrupt: ignored

In [0]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [0]:
# Loss and Accuracy for Training Dataset
plt_loss = results.history["loss"]
plt_acc = results.history["acc"]
# Loss and Accuracy for Validation (Test) Dataset
plt_val_loss = results.history["val_loss"]
plt_val_acc = results.history["val_acc"]

num_epochs = len(plt_val_acc)
plt_epoch = np.arange(1, num_epochs+1, 1, dtype=int)

In [0]:
plt.figure(1, figsize=(10, 6))
plt.plot(plt_epoch, plt_acc, color='c', linewidth=4, label='Training Dataset')
plt.plot(plt_epoch, plt_val_acc, color='m', linewidth=4, label='Test Dataset')
plt.xlim(1, epochs)
plt.ylim(0.41, 1.0)
plt.xticks(np.arange(0, num_epochs+1, 10, dtype=int))
plt.legend(loc='lower right', fontsize=15)
plt.xlabel(r'$Epochs$', fontsize=15)
plt.ylabel(r'$Accuracy$', fontsize=15)
plt.grid(True)
plt.show()

In [0]:
plt.figure(1, figsize=(10, 6))
plt.plot(plt_epoch, plt_val_loss, color='m', linewidth=4, label='Test Dataset')
plt.plot(plt_epoch, plt_loss, color='c', linewidth=4, label='Training Dataset')
plt.xlim(1, epochs)
plt.xticks(np.arange(0, num_epochs+1, 10, dtype=int))
plt.legend(loc='upper left', fontsize=15)
plt.xlabel(r'$Epochs$', fontsize=15)
plt.ylabel(r'$Loss$', fontsize=15)
plt.grid(True)
plt.show()